# Basic ML model

In [1]:
import pandas as pd


In [13]:
bv = pd.read_csv("../data/external/bill_version.csv", sep=";", encoding="latin1", parse_dates=True).rename(columns={'id': "version_id"})
bills = pd.read_csv("../data/external/bill.csv", sep=";", encoding="latin1", parse_dates=True).rename(columns={'id': 'bill_id'})
bv_features = pd.read_csv("../data/derived/basic_version_features.csv", sep=",", encoding="latin1", parse_dates=True).rename(columns={'id': 'version_id'})
partisan_lean = pd.read_csv("../data/derived/partisan_lean.csv", sep=",", encoding="latin1", parse_dates=True)


In [3]:
bills.head(n=1)

,bill_id,os_bill_id,name,state_id,chamber_id,session_id,title,created_at,updated_at,first_action,last_action,passed_lower,passed_upper,signed,division_id,bill_prefix,bill_number
0,830501,NaN,PC 101,40,2.0,475,Para ordenar a todo patrono notificar por escr...,2017-01-06 16:46:38,2020-06-26 03:32:36,2017-01-02 04:00:00,2020-06-25 04:00:00,NaN,NaN,NaN,156409,PC,101


In [4]:
bv.head(n=1)

,version_id,name,mimetype,os_doc_id,url,bill_id,mimetype_id,bill_printing_id,document_type_id,filesize,plain_filesize,bypass,created_at,updated_at
0,1528347,eh,text/html,NaN,https://www.gpo.gov/fdsys/pkg/BILLS-115hres3eh...,827210,2.0,284614.0,NaN,NaN,NaN,0,NaN,2019-01-04 04:00:04


In [10]:
df = bv[['version_id', 'bill_id']].merge(bills[['bill_id', 'chamber_id', 'session_id', 'signed']])
df2 = df.merge(partisan_lean[['session_id', 'chamber_id', 'partisan_lean']], on=['session_id', 'chamber_id'])

In [14]:
df_ml = df2.merge(bv_features, on='version_id')


In [24]:
df_ml.loc[df_ml.signed.isna(), 'signed'] = 0
df_ml.loc[df_ml.signed != 0, 'signed'] = 1

In [25]:
df_ml.head()

,version_id,bill_id,chamber_id,session_id,signed,partisan_lean,text_length
0,2119301,1092981,1.0,562,0,0.236118,43841
1,2708948,1092981,1.0,562,0,0.236118,53832
2,2708949,1092981,1.0,562,0,0.236118,60077
3,2708950,1092981,1.0,562,0,0.236118,7175
4,2708951,1092981,1.0,562,0,0.236118,961


In [26]:
df_ml_X = df_ml[['session_id', 'partisan_lean', 'text_length']]
df_ml_Y = df_ml[['signed']]

In [27]:
from sklearn.preprocessing import OneHotEncoder